In [0]:
-- Pivots Causes of death into a long format
--
-- Excludes records with blank DEC_CONF_NHS_NUMBER_CLEAN_DEID
--
-- there does not seem to be a record level identifier in the
-- mortality data. Im guessing that if there is no DEC_CONF_NHS_NUMBER_CLEAN_DEID
-- on a row, then that means it did match some central spine of "NHS people".
-- Anyway - the blanks may be useful for mortality statistics (although without -- LSOA, best of luck with that). Otherwise, excluding them means you are 
-- potentially under counting in a population sense, by omitting those never 
-- treated

--DROP TABLE IF EXISTS hive_metastore.dsa_484452_h8s1l_collab.peolc_deaths_cod_long;

CREATE OR REPLACE TEMPORARY VIEW peolc_deaths_cod_long AS
WITH dths AS (	
  SELECT 
    DEC_CONF_NHS_NUMBER_CLEAN_DEID
    ,NEO_NATE_FLAG
    ,CAST(LEFT(REG_DATE_OF_DEATH, 4) AS INT) AS xYear
    ,STACK(16, 
		0, S_UNDERLYING_COD_ICD10,
    1, S_COD_CODE_1,
    2, S_COD_CODE_2,
    3, S_COD_CODE_3,
    4, S_COD_CODE_4,
    5, S_COD_CODE_5,
    6, S_COD_CODE_6,
    7, S_COD_CODE_7,
    8, S_COD_CODE_8,
    9, S_COD_CODE_9,
    10, S_COD_CODE_10,
    11, S_COD_CODE_11,
    12, S_COD_CODE_12,
    13, S_COD_CODE_13,
    14, S_COD_CODE_14,
    15, S_COD_CODE_15) AS (cod_index, icd )
  FROM hive_metastore.dars_nic_484452_h8s1l.deaths_dars_nic_484452_h8s1l
	WHERE DEC_CONF_NHS_NUMBER_CLEAN_DEID IS NOT NULL
  -- LIMIT 1000
	)

SELECT
  DEC_CONF_NHS_NUMBER_CLEAN_DEID
  ,NEO_NATE_FLAG
  ,xYear
  ,icd
  ,cod_index
  ,CASE 
      WHEN cod_index=0 THEN 1 ELSE 0 END   AS ucod
FROM dths
WHERE cod_index=0 OR ((cod_index>0) AND (icd IS NOT NULL) );

-----

SELECT * FROM peolc_deaths_cod_long
LIMIT 500
